In [ ]:
import os
import shutil

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pretty_midi
import IPython.display as ipd
import libfmp.c1

from utils.data import *

In [ ]:
midi_note_df = pd.read_csv('./data/midi_to_notes.csv', index_col=0)
midi_note_df

In [ ]:
path = './data/theorytab/MIDI/t/the-weeknd/cant-feel-my-face/pre-chorus-and-chorus_symbol_key.mid'
midi_data = pretty_midi.PrettyMIDI(path)
df = midi_to_df(midi_data)

In [ ]:
for key in midi_data.key_signature_changes:
    print(key.key_number, key.time)

In [ ]:
df

In [ ]:
print(f'Key of song: {pretty_midi.key_number_to_key_name(midi_data.key_signature_changes[0].key_number)}')
print(f'Time signature: {midi_data.time_signature_changes[0].numerator}/{midi_data.time_signature_changes[0].denominator}')
print(f'Event time in seconds of each beat: \n {midi_data.get_beats()}')
print(f'Beat start in seconds: {midi_data.estimate_beat_start()}')
print(f'Beat start / beat duration: {midi_data.estimate_beat_start() / midi_data.get_beats()[1]}')
print(f'Beats per minute based on pretty_midi.estimate_tempo: {midi_data.estimate_tempo()}')
print(f'Beats per minute based on 60s/beat duration: {60 / midi_data.get_beats()[1]}')

In [ ]:
df.loc[:, ['Start', 'End']] /= midi_data.get_beats()[1]
df.loc[:, ['Start', 'End']] *= 0.5
df

In [ ]:
fs = 44100
audio_data = df_to_midi(df).fluidsynth(fs=fs, sf2_path='./data/soundfonts/Steinway_Grand_Piano_1.2.sf2')
ipd.Audio(audio_data, rate=fs)

In [ ]:
df = add_octave_and_note(df, midi_note_df)

In [ ]:
midi_data2 = df_to_midi(df)

In [ ]:
midi_data2.instruments

In [ ]:
df2 = midi_to_df(midi_data2)
df2

# Play audio

In [ ]:
fs = 44100
audio_data = midi_data.fluidsynth(fs=fs, sf2_path='./data/soundfonts/Steinway_Grand_Piano_1.2.sf2')
ipd.Audio(audio_data, rate=fs)

In [ ]:
fs = 22050
audio_data = midi_data.fluidsynth(fs=fs, sf2_path='./data/soundfonts/Steinway_Grand_Piano_1.2.sf2')
ipd.Audio(audio_data, rate=fs)

In [ ]:
score = midi_to_list(midi_data)
libfmp.c1.visualize_piano_roll(score, figsize=(16, 9), velocity_alpha=True);

# Check keys of different midi files

In [ ]:
key_counts = np.zeros(24, dtype=int)
get_key_and_sort_files_to_dir('./data/theorytab/MIDI/', key_counts)
fig = plt.figure(figsize=(12, 6))
plt.bar(x=[get_key_note(i, tight=True) for i in range(24)], height=key_counts)
plt.tight_layout()

# Change all single key songs to key of C Major

In [ ]:
remove_empty('./data/single_key/')

In [ ]:
def change_key(directory: str, n_midi_files: int=0):
    if n_midi_files > 0: # multiple midi files for same song
        pitches = []
        notes = []
        for entry in os.scandir(directory):
            if entry.is_file() and entry.name.endswith('.mid'):
                midi_data = pretty_midi.PrettyMIDI(entry.path)
                # key_signature = midi_data.key_signature_changes
                df = midi_to_df(midi_data)
                unique_pitches = np.unique(df['Pitch'].values)
                pitches += list(unique_pitches)
                notes += [pretty_midi.note_number_to_name(p) for p in unique_pitches]
        pitches = set(pitches)
        notes = set(notes)
        print(f'{directory}: {notes}')
    else:
        for entry in os.scandir(directory):
            if entry.is_dir():
                n_midi_files = len([name for name in os.listdir(entry.path) if name.endswith('.mid')])
                change_key(entry.path, n_midi_files)

In [ ]:
change_key('./data/single_key/4/')

In [ ]:
path = './data/theorytab/MIDI/p/prince/purple-rain/verse_symbol_key.mid'
midi_data = pretty_midi.PrettyMIDI(path)
df = midi_to_df(midi_data)
df